**Objective : TO perform RAG-SQL on Movies Dataset**

### Installing all dependencies

In [1]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

### Setting up Environment

In [2]:
OPENAI_API_KEY = "ENTER_API_KEY_HERE"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\gkris\AppData\Local\Temp\ipykernel_19460\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


The API key is provided when you sign up for OpenAI's services and is required to access their language models (like GPT-3 or GPT-4). This key will be used later to interact with OpenAI’s language models through the ChatOpenAI class."temperature=0" This controls the "creativity" or randomness of the language model

### Database Setup

In [3]:
host = 'localhost'
port = '3306'
username = 'root'
#password = ''
database_schema = 'banksql'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
#mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['p6_uk_bank_customers'],sample_rows_in_table_info=4)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True,top_k = 2500)

This creates an instance of SQLDatabase, which establishes a connection to the MySQL database using the previously constructed mysql_uri. "mysql_uri" is the URI containing the credentials and details to connect to the MySQL database.db_chain" creates a SQLDatabaseChain, which is responsible for converting natural language queries into SQL queries, running them against the MySQL database, and retrieving results. The chain uses both the language model (llm) and the database connection (db).

### Function to Retrieve and Clean Database Query Result

In [4]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

The retrieve_from_db function processes a query by passing it through a db_chain. It extracts the 'result' from the chain's output and removes any leading/trailing whitespace using .strip(). Finally, it returns the cleaned result as a string.

### Function to Generate Bank Customer Details Information Response Based on User Query

In [15]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a bank SQL expert specializing in account schemes.
        Your task is to answer users' questions by providing relevant information from a database of bank applicants.
        The database contains the following information for each applicant:
        - Customer ID : To know the unique identifier of customer from database.
        - Name : To know the customer name
        - Surname : To differentiate the customers having same name.
        - Gender : To know whether customer is male or female.
        - Age : To know current age of customers.
        - Region : To know The specific residence of customer
        - Job Classification : To know specific postion of customer in terms of job wise.
        - Date Joined : To know when he joined the bank.
        - Balance : To know the current balance of customers in the bank.
        """
    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

The generate function first retrieves the relevant database context using the retrieve_from_db function. It then formats a prompt for a language model with a system message that provides the context of loan applicants' data, followed by the user's query. The language model is then invoked to generate and return a response based on the input query and context.

### Generating Queries

In [ ]:
#1
generate("provide me the names of customers from england")

C:\Users\gkris\AppData\Local\Temp\ipykernel_19460\3606437300.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
provide me the names of customers from england
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Region` = 'England'
LIMIT 2500;
SQLResult: [('Simon',), ('Liam',), ('Deirdre',), ('Dorothy',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Ryan',), ('Nicholas',), ('William',), ('Audrey',), ('Jennifer',), ('Alison',), ('Luke',), ('Megan',), ('Jake',), ('Karen',), ('Yvonne',), ('Sophie',), ('Robert',), ('Edward',), ('Diane',), ('Dorothy',), ('Simon',), ('Charles',), ('Sophie',), ('Kimberly',), ('Madeleine',), ('Irene',), ('Cameron',), ('Liam',), ('Donna',), ('Piers',), ('Phil',), ('Ryan',), ('Kylie',), ('Fiona',), ('Kevin',), ('Bernadette',), ('Frank',), ('Diane',), ('David',), ('Joan',), ('Neil',), ('Paul',), ('Dylan',), ('Pippa',), ('Matt',), ('Jack',), ('Jasmine',), ('Michelle',), ('Wanda',), ('Blake',), ('Jane',), ('Dan',), ('Lisa',), ('Rebecca',), ('Joshua',), ('Heather',), ('Maria',), ('Alison',), ('Amy',), ('Vanessa',), ('Andrea',), ('

C:\Users\gkris\AppData\Local\Temp\ipykernel_19460\1510247937.py:32: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content


'The names of customers from England are: Simon, Liam, Deirdre, Dorothy, Dominic, Benjamin, Ryan, Nicholas, William, Audrey, Jennifer, Alison, Luke, Megan, Jake, Karen, Yvonne, Sophie, Robert, Edward, Diane, Charles, Kimberly, Madeleine, Irene, Cameron, Donna, Piers, Phil, Kylie, Fiona, Kevin, Bernadette, Frank, David, Joan, Neil, Paul, Dylan, Pippa, Matt, Jack, Jasmine, Michelle, Wanda, Blake, Jane, Dan, Lisa, Rebecca, Joshua, Heather, Maria, Amy, Vanessa, Andrea, Ian, Jacob, Harry, John, Jonathan, Emma, Virginia, Peter, Anna, Ruth, Owen, Christopher, Sarah, Richard, Nicola, Angela, Sam, Sean, Steven, Andrew, Madeleine, Connor, Justin, Elizabeth, Tim, Anthony, Michael, Lucas, Diana, Amanda, Natalie, Hannah, Penelope, Max, Wendy, Thomas, Austin, Christian, Cameron, Trevor, Stewart, Brian, Carol, Sonia, Anne, Sue, Jan, Lillian, Gabrielle, Samson, Hans, Ludwig, Sam, Samuel, Ivan, Hans, Samson, Jackson, Scott.'

In [7]:
#2
generate("Provide the count of male people")



> Entering new SQLDatabaseChain chain...
Provide the count of male people
SQLQuery:SELECT COUNT(`Customer ID`) AS Male_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Male';
SQLResult: [(2165,)]
Answer:There are 2165 male people in the database.
> Finished chain.


'The count of male people in the database is 2165.'

In [8]:
#3
generate(" provide me the count of female from wales")



> Entering new SQLDatabaseChain chain...
 provide me the count of female from wales
SQLQuery:SELECT COUNT(`Customer ID`) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales';
SQLResult: [(282,)]
Answer:There are 282 females from Wales.
> Finished chain.


'The count of females from Wales in the database is 282.'

In [ ]:
#4
generate("provide the count of customers with white collar jobs")



> Entering new SQLDatabaseChain chain...
provide the count of customers with white collar jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 customers with white collar jobs.
> Finished chain.


'The count of customers with white collar jobs is 1955.'

In [ ]:
#5
generate("provide me the count of customers who are having white collar jobs") 



> Entering new SQLDatabaseChain chain...
provide me the count of customers who are having white collar jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 customers who have white collar jobs.
> Finished chain.


'The count of customers who are having white collar jobs is 1955.'

In [ ]:
#6
generate("provide me the count of customers whose bank balance is less more than 5000")



> Entering new SQLDatabaseChain chain...
provide me the count of customers whose bank balance is less more than 5000
SQLQuery:SELECT COUNT(`Customer ID`) AS `Customer Count`
FROM p6_uk_bank_customers
WHERE `Balance` > 5000
LIMIT 2500;
SQLResult: [(3690,)]
Answer:There are 3690 customers whose bank balance is more than 5000.
> Finished chain.


'The count of customers whose bank balance is more than 5000 is 3690.'

In [ ]:
#7
generate ("provide me the list of people who are having white collar jobs and age is above 35")



> Entering new SQLDatabaseChain chain...
provide me the list of people who are having white collar jobs and age is above 35
SQLQuery:SELECT `Name`, `Surname`, `Age`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar' AND `Age` > 35
LIMIT 2500;
SQLResult: [('Liam', 'Brown', 46), ('Dominic', 'Parr', 42), ('Dominic', 'Lewis', 40), ('Benjamin', 'Grant', 39), ('Nicholas', 'Newman', 42), ('Deirdre', 'McDonald', 41), ('Jennifer', 'Hughes', 38), ('Rachel', 'McGrath', 37), ('Luke', 'Wilkins', 42), ('Yvonne', 'Thomson', 36), ('Charles', 'Nolan', 39), ('Liam', 'Gibson', 40), ('Piers', 'Morgan', 44), ('Ryan', 'White', 47), ('Kevin', 'Gibson', 47), ('Bernadette', 'Gibson', 39), ('Hannah', 'White', 36), ('Frank', 'Brown', 38), ('Joan', 'Buckland', 36), ('Dylan', 'Fraser', 42), ('Pippa', 'Buckland', 38), ('Matt', 'McGrath', 39), ('Maria', 'Johnston', 36), ('Emma', 'Piper', 44), ('Nicola', 'Powell', 55), ('Stephen', 'Ross', 48), ('Ian', 'Slater', 37), ('Harry', 'Short', 50), ('Benj

"SELECT Name, Surname\nFROM bank_applicants\nWHERE Job_Classification = 'White Collar' AND Age > 35;"

In [ ]:
#8
generate(" provide me the count of females who are having white collar jobs")



> Entering new SQLDatabaseChain chain...
 provide me the count of females who are having white collar jobs
SQLQuery:SELECT COUNT(*) AS FemaleWhiteCollarCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND `Job Classification` = 'White Collar';
SQLResult: [(1010,)]
Answer:There are 1010 females who have white collar jobs.
> Finished chain.


'The count of females who are having white collar jobs is 1010.'

In [ ]:
#9
generate("provide me the count of peopel who are from wales and doing white collar jobs ")



> Entering new SQLDatabaseChain chain...
provide me the count of peopel who are from wales and doing white collar jobs 
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE Region = 'Wales' AND `Job Classification` = 'White Collar';
SQLResult: [(305,)]
Answer:There are 305 people from Wales who are doing white collar jobs.
> Finished chain.


'The count of people who are from Wales and doing white collar jobs is 305.'

In [ ]:
#10
generate("provide average age of people with white collar jobs")



> Entering new SQLDatabaseChain chain...
provide average age of people with white collar jobs
SQLQuery:SELECT AVG(`Age`) AS `Average Age`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar'
SQLResult: [(Decimal('35.2793'),)]
Answer:The average age of people with white collar jobs is 35.2793.
> Finished chain.


'The average age of people with white collar jobs is 35.2793.'

In [ ]:
#11
generate("provide me the names of customers who are having maximum balance")



> Entering new SQLDatabaseChain chain...
provide me the names of customers who are having maximum balance
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
ORDER BY `Balance` DESC
LIMIT 1;
SQLResult: [('Dorothy',)]
Answer:Dorothy
> Finished chain.


'The names of customers who are having the maximum balance are:\n1. Dorothy'

In [ ]:
#12
generate("provie me the maximum balance, name of the person and job classification")



> Entering new SQLDatabaseChain chain...
provie me the maximum balance, name of the person and job classification
SQLQuery:SELECT MAX(`Balance`) AS `Max Balance`, `Name`, `Job Classification`
FROM p6_uk_bank_customers;
SQLResult: [(Decimal('183467.70'), 'Simon', 'White Collar')]
Answer:The maximum balance in the bank is £183,467.70, and the person with this balance is Simon who is classified as White Collar.
> Finished chain.


'The maximum balance in the bank is £183,467.70, and the person with this balance is Simon who is classified as White Collar.'

seperate queries

In [21]:
#13
generate("provide me the names of applicants who have same name")



> Entering new SQLDatabaseChain chain...
provide me the names of applicants who have same name
SQLQuery:SELECT `Name`, COUNT(`Name`) AS NameCount
FROM p6_uk_bank_customers
GROUP BY `Name`
HAVING COUNT(`Name`) > 1
ORDER BY NameCount DESC
LIMIT 2500;
SQLResult: [('Jason', 42), ('John', 39), ('Blake', 38), ('Dominic', 37), ('Yvonne', 36), ('Lucas', 36), ('David', 35), ('Alan', 35), ('Ruth', 34), ('Leonard', 34), ('Boris', 33), ('Matt', 33), ('Owen', 33), ('Paul', 33), ('Peter', 33), ('Joshua', 33), ('Brian', 32), ('Ian', 32), ('Dylan', 32), ('Adam', 32), ('Deirdre', 32), ('Tim', 32), ('James', 31), ('Chloe', 31), ('Christopher', 31), ('Jake', 31), ('Gavin', 31), ('Harry', 31), ('Piers', 31), ('Jasmine', 30), ('Amy', 30), ('William', 30), ('Kevin', 30), ('Sebastian', 30), ('Gordon', 30), ('Heather', 29), ('Simon', 29), ('Nicholas', 29), ('Sam', 29), ('Sean', 29), ('Fiona', 29), ('Isaac', 29), ('Penelope', 29), ('Keith', 29), ('Amanda', 28), ('Victor', 28), ('Warren', 28), ('Nathan', 28),

'The names of applicants who have the same name are: Alan, Anna, Anne, Carol, Christian, Colin, Dan, Diana, Diane, Emma, Eric, Grace, Hannah, Jane, Joan, John, Joseph, Julia, Justin, Karen, Katherine, Keith, Kimberly, Laura, Lauren, Leah, Lily, Lisa, Luke, Maria, Mary, Megan, Michelle, Nathan, Neil, Olivia, Paul, Peter, Rachel, Rebecca, Rose, Ryan, Sam, Samantha, Sarah, Sean, Simon, Sonia, Sophie, Stephen, Sue, Theresa, Thomas, Tim, Victoria, Wendy.'

In [ ]:
#14
generate("provide me the names of applicants who are having same bank balance ")



> Entering new SQLDatabaseChain chain...
provide me the names of applicants who are having same bank balance 
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Balance` IN (
    SELECT `Balance`
    FROM p6_uk_bank_customers
    GROUP BY `Balance`
    HAVING COUNT(*) > 1
)
ORDER BY `Balance` DESC
LIMIT 2500;
SQLResult: [('Karen',), ('Kimberly',), ('Matt',), ('Jessica',)]
Answer:Karen, Kimberly, Matt, and Jessica are the names of applicants who are having the same bank balance.
> Finished chain.


'Karen, Kimberly, Matt, and Jessica'

In [ ]:
#15
generate ("provide me the count of applicants who are having same name and having same surname")



> Entering new SQLDatabaseChain chain...
provide me the count of applicants who are having same name and having same surname
SQLQuery:SELECT `Name`, `Surname`, COUNT(*) AS `Count`
FROM p6_uk_bank_customers
GROUP BY `Name`, `Surname`
HAVING COUNT(*) > 1
ORDER BY `Count` DESC;
SQLResult: [('David', 'McLean', 2), ('Alexander', 'Hudson', 2), ('Isaac', 'Quinn', 2), ('Dylan', 'Parr', 2), ('Jake', 'Ball', 2), ('John', 'Payne', 2), ('Lucas', 'Abraham', 2), ('Dominic', 'Taylor', 2), ('John', 'Campbell', 2), ('Luke', 'Harris', 2), ('Nicholas', 'Allan', 2), ('Alan', 'Murray', 2), ('Amy', 'Graham', 2), ('Isaac', 'Gill', 2), ('Justin', 'Wallace', 2), ('Christopher', 'Langdon', 2), ('Peter', 'Bailey', 2), ('Harry', 'Reid', 2), ('Victor', 'Gray', 2), ('Tim', 'Marshall', 2), ('Christian', 'Miller', 2), ('Tim', 'Sanderson', 2), ('Victor', 'Black', 2), ('Ian', 'Terry', 2), ('David', 'Cameron', 2), ('Christian', 'Hemmings', 2), ('Ella', 'Knox', 2), ('Vanessa', 'McLean', 2), ('Julian', 'Hunter', 2), ('J

'To provide you with the count of applicants who have the same name and surname, I will need to run the following SQL query:\n\n```sql\nSELECT Name, Surname, COUNT(*) AS Count\nFROM BankApplicants\nGROUP BY Name, Surname\nHAVING COUNT(*) > 1;\n```\n\nThis query will give us the count of applicants who share the same name and surname.'

In [ ]:
#16
generate("provide me the count of people who are having same name and having same job application")



> Entering new SQLDatabaseChain chain...
provide me the count of people who are having same name and having same job application
SQLQuery:SELECT `Name`, `Job Classification`, COUNT(*) AS `Count`
FROM p6_uk_bank_customers
GROUP BY `Name`, `Job Classification`
HAVING COUNT(*) > 1
ORDER BY `Count` DESC;
SQLResult: [('Blake', 'Blue Collar', 26), ('Ruth', 'White Collar', 25), ('Yvonne', 'White Collar', 22), ('Paul', 'Blue Collar', 20), ('Jason', 'Blue Collar', 19), ('Deirdre', 'White Collar', 19), ('David', 'White Collar', 19), ('Bernadette', 'White Collar', 19), ('Chloe', 'White Collar', 19), ('Heather', 'White Collar', 18), ('Anna', 'White Collar', 18), ('Lucas', 'Blue Collar', 18), ('John', 'White Collar', 18), ('Dominic', 'White Collar', 18), ('Fiona', 'White Collar', 17), ('Matt', 'White Collar', 17), ('James', 'White Collar', 17), ('Oliver', 'White Collar', 16), ('Evan', 'Blue Collar', 16), ('Boris', 'White Collar', 16), ('Liam', 'White Collar', 16), ('Grace', 'White Collar', 16), (

'To provide you with the count of people who have the same name and job classification, the following SQL query can be used:\n\n```sql\nSELECT Name, JobClassification, COUNT(*) AS CountOfPeople\nFROM BankApplicants\nGROUP BY Name, JobClassification\nHAVING COUNT(*) > 1;\n```\n\nThis query will group the applicants by their name and job classification, and then count the number of occurrences for each unique combination. The `HAVING COUNT(*) > 1` condition ensures that only the combinations with more than one occurrence are included in the result.'

In [25]:
#17
generate("provide me the names of female who are hvaing white collare jobs and having balance above than 50000")



> Entering new SQLDatabaseChain chain...
provide me the names of female who are hvaing white collare jobs and having balance above than 50000
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Gender` = 'Female' AND `Job Classification` = 'White Collar' AND `Balance` > 50000
LIMIT 2500;
SQLResult: [('Ruth',), ('Deirdre',), ('Joan',), ('Pippa',), ('Jasmine',), ('Rebecca',), ('Andrea',), ('Nicola',), ('Elizabeth',), ('Hannah',), ('Mary',), ('Jennifer',), ('Karen',), ('Fiona',), ('Yvonne',), ('Wanda',), ('Fiona',), ('Maria',), ('Theresa',), ('Olivia',), ('Penelope',), ('Felicity',), ('Jessica',), ('Nicola',), ('Felicity',), ('Tracey',), ('Anna',), ('Jasmine',), ('Ruth',), ('Wendy',), ('Amy',), ('Pippa',), ('Wendy',), ('Amanda',), ('Amelia',), ('Chloe',), ('Felicity',), ('Anne',), ('Lillian',), ('Gabrielle',), ('Madeleine',), ('Diana',), ('Lillian',), ('Molly',), ('Jasmine',), ('Heather',), ('Penelope',), ('Diane',), ('Jan',), ('Julia',), ('Katherine',), ('Nicola',), ('Chloe',), ('B

'The names of females who have white-collar jobs and a balance above 50000 are Ruth, Deirdre, Joan, Pippa, Jasmine, Rebecca, Andrea, Nicola, Elizabeth, Hannah, Mary, Jennifer, Karen, Fiona, Yvonne, Wanda, Maria, Theresa, Olivia, Penelope, Felicity, Jessica, Tracey, Anna, Wendy, Amy, Amanda, Amelia, Chloe, Anne, Lillian, Gabrielle, Madeleine, Diana, Molly, Heather, Diane, Jan, Julia, Katherine, Bella, Melanie, Megan, Rose, Natalie, Sally, Victoria, Sophie, Grace, Ella, Una, Emma, Sonia, Caroline, Ava, Emily, Dorothy, Zoe, Leah, Jane, Abigail, Samantha, Donna, Lily, Angela, Virginia, Vanessa, Kylie, Bernadette, Irene, Audrey, Michelle, Lisa, Stephanie, Claire, Carolyn, Alison, Carol, Sue, Kimberly, Faith, Jessica, Rachel, and Alexandra.'

In [ ]:
#18
generate("provide me the count of applicants based on their job classification")



> Entering new SQLDatabaseChain chain...
provide me the count of applicants based on their job classification
SQLQuery:SELECT `Job Classification`, COUNT(`Customer ID`) AS `Applicant Count`
FROM p6_uk_bank_customers
GROUP BY `Job Classification`
ORDER BY `Applicant Count` DESC;
SQLResult: [('White Collar', 1955), ('Blue Collar', 1049), ('Other', 1010)]
Answer:There are 1955 applicants in the "White Collar" job classification, 1049 in the "Blue Collar" job classification, and 1010 in the "Other" job classification.
> Finished chain.


'Based on the provided information, the count of applicants based on their job classification is as follows:\n- "White Collar": 1955 applicants\n- "Blue Collar": 1049 applicants\n- "Other": 1010 applicants'

In [27]:
#19
generate("provide me the count of people who based on their balance")



> Entering new SQLDatabaseChain chain...
provide me the count of people who based on their balance
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`, 
	CASE
		WHEN `Balance` < 50000 THEN 'Low Balance'
		WHEN `Balance` >= 50000 AND `Balance` < 100000 THEN 'Medium Balance'
		ELSE 'High Balance'
	END AS `Balance Category`
FROM p6_uk_bank_customers
GROUP BY `Balance Category`
ORDER BY `Count` DESC;
SQLResult: [(2736, 'Low Balance'), (1105, 'Medium Balance'), (173, 'High Balance')]
Answer:There are 2736 people with Low Balance, 1105 people with Medium Balance, and 173 people with High Balance based on their balance.
> Finished chain.


'Based on the provided information, the count of people based on their balance is as follows:\n- 2736 people with Low Balance\n- 1105 people with Medium Balance\n- 173 people with High Balance'

In [28]:
#20
generate("provide me the names of people whose first letter is J")



> Entering new SQLDatabaseChain chain...
provide me the names of people whose first letter is J
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Name` LIKE 'J%'
LIMIT 2500;
SQLResult: [('Jasmine',), ('Jason',), ('Jennifer',), ('Jake',), ('Jake',), ('Jason',), ('Joan',), ('Jack',), ('Jasmine',), ('Jane',), ('Joshua',), ('Joe',), ('Jacob',), ('John',), ('Jonathan',), ('James',), ('Jacob',), ('Justin',), ('John',), ('Jason',), ('Jane',), ('James',), ('Jennifer',), ('Jasmine',), ('Jessica',), ('Jessica',), ('John',), ('Justin',), ('James',), ('Julian',), ('Justin',), ('James',), ('Jake',), ('Joe',), ('Joseph',), ('John',), ('Jessica',), ('John',), ('Joanne',), ('Jonathan',), ('Jake',), ('Jasmine',), ('Joseph',), ('Jake',), ('Joshua',), ('Joan',), ('Joshua',), ('John',), ('Jasmine',), ('Joe',), ('Joseph',), ('Julia',), ('Jacob',), ('Jake',), ('Joshua',), ('Jan',), ('Justin',), ('Jake',), ('Jasmine',), ('James',), ('Joan',), ('James',), ('Joan',), ('Justin',), ('Julian',), ('Jacob',

'Jason, Jennifer, Jake, Joan, Jack, Jane, Joshua, Joe, Jacob, John, Jonathan, James, Justin, Julian, Julia, Jessica, Joanne, Joseph, Jan'